In [26]:
import sys
import os
import pandas as pd

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
from src.data_tool.data_providers.connect_wrds import get_wrds_data, get_security_daily

In [27]:
from googleapiclient.discovery import build
from google.oauth2.service_account import Credentials

In [28]:
import gspread

# gc = gspread.service_account(filename='stocksflags-ec5c40f2f2ee.json')
gc = gspread.service_account(filename='rational-diode-456114-m0-79243525427e.json')


ginzu_spreadsheet_link = 'https://docs.google.com/spreadsheets/d/1xclQf2xrgw0swp2CRwE25OBaEhQdDTOkm8VVkTcpVks/edit?usp=sharing'
pricer = gc.open_by_url(ginzu_spreadsheet_link)
input_worksheet = pricer.get_worksheet(0)
valuation_worksheet = pricer.get_worksheet(1)

In [ ]:
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
SERVICE_ACCOUNT_FILE = 'rational-diode-456114-m0-79243525427e.json'

spreadsheet_id = '1xclQf2xrgw0swp2CRwE25OBaEhQdDTOkm8VVkTcpVks'
sheet_name = 'Input sheet'

creds = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
service = build('sheets', 'v4', credentials=creds)

In [16]:
def get_dropdown_options(spreadsheet_id: str, sheet_name: str, cell: str) -> list:
    """
    Given a spreadsheet ID, sheet name, and cell (e.g., 'B7'),
    returns the dropdown options (either hardcoded or from a range).
    """
    range_name = f"'{sheet_name}'!{cell}"
    
    try:
        response = service.spreadsheets().get(
            spreadsheetId=spreadsheet_id,
            ranges=[range_name],
            includeGridData=True,
            fields='sheets.data.rowData.values.dataValidation'
        ).execute()

        validation = response['sheets'][0]['data'][0]['rowData'][0]['values'][0]['dataValidation']
        condition = validation['condition']
        dropdown_values = [v['userEnteredValue'] for v in condition.get('values', [])]

        dropdown_ref = dropdown_values[0]
        if dropdown_ref.startswith("="):
            raw_ref = dropdown_ref.lstrip("=").replace("'", "")
            ref_sheet, ref_range = raw_ref.split("!")

            country_resp = service.spreadsheets().values().get(
                spreadsheetId=spreadsheet_id,
                range=f"{ref_sheet}!{ref_range}"
            ).execute()

            return [row[0].strip() for row in country_resp.get("values", []) if row and row[0].strip()]
        else:
            return dropdown_values

    except (KeyError, IndexError):
        print(f"No dropdown (data validation) found at {sheet_name}!{cell}.")
        return []


## Date of Valuation and Company Name

In [ ]:
date_of_valuation = input('What is the date of the valuation? (Month-Day, e.g. Mar-15)')
company_name = input('What is the company name?')
input_worksheet.update('B3', [[date_of_valuation]])
input_worksheet.update('B4', [[company_name]])

/var/folders/zc/mk397vzd3rndczvtnq2xwwzr0000gn/T/ipykernel_97585/1735246152.py:3: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  input_worksheet.update('B3', [[date_of_valuation]])
/var/folders/zc/mk397vzd3rndczvtnq2xwwzr0000gn/T/ipykernel_97585/1735246152.py:4: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  input_worksheet.update('B4', [[company_name]])


{'spreadsheetId': '1xclQf2xrgw0swp2CRwE25OBaEhQdDTOkm8VVkTcpVks',
 'updatedRange': "'Input sheet'!B4",
 'updatedRows': 1,
 'updatedColumns': 1,
 'updatedCells': 1}

## Country of Incorporation, Industry (Global) and Industry (US)

In [19]:
country_names = get_dropdown_options(spreadsheet_id, sheet_name, 'B7')
us_industry = get_dropdown_options(spreadsheet_id, sheet_name, 'B8')
global_industry = get_dropdown_options(spreadsheet_id, sheet_name, 'B9')

In [ ]:
country_of_incorporation = input('What is the country of incorporation?')
us_industry = input('What is the industry in the US market?')
global_industry = input('What is the industry in the global market?')
input_worksheet.update('B7', [[country_of_incorporation]])
input_worksheet.update('B8', [[us_industry]])
input_worksheet.update('B9', [[global_industry]])

## Years Since Last 10K

In [ ]:
years_since_last_10k = input('How many years since the last 10K?')
input_worksheet.update('D11',[[years_since_last_10k]])

## Revenues (Most Recent 12 Months, Last 10K)

In [ ]:
current_revenues = input('What is the current revenues?')
last_10k_revenues = input('What is the last 10K revenues?')
input_worksheet.update('B11', [[current_revenues]])
input_worksheet.update('C11', [[last_10k_revenues]])

## Operating Income / EBIT (Most Recent 12 Months, Last 10K)

In [ ]:
current_operating_income = input('What is the current operating income?')
last_10k_operating_income = input('What is the last 10K operating income?')
input_worksheet.update('B12', [[current_operating_income]])
input_worksheet.update('C12', [[last_10k_operating_income]])

## Interest Expense (Most Recent 12 Months, Last 10K)

In [ ]:
current_interest_expense = input('What is the current interest expense?')
last_10k_interest_expense = input('What is the last 10K interest expense?')
input_worksheet.update('B13', [[current_interest_expense]])
input_worksheet.update('C13', [[last_10k_interest_expense]])

## Book Value of Equity (Most Recent 12 Months, Last 10K)

In [ ]:
current_book_value_of_equity = input('What is the current book value of equity?')
last_10k_book_value_of_equity = input('What is the last 10K book value of equity?')
input_worksheet.update('B14', [[current_book_value_of_equity]])
input_worksheet.update('C14', [[last_10k_book_value_of_equity]])

## Book Value of Debt (Most Recent 12 Months, Last 10K)

In [ ]:
current_book_value_of_debt = input('What is the current book value of debt?')
last_10k_book_value_of_debt = input('What is the last 10K book value of debt?')
input_worksheet.update('B15', [[current_book_value_of_debt]])
input_worksheet.update('C15', [[last_10k_book_value_of_debt]])

## R&D Expenses to Capitalize + Operating Lease Commitments

In [20]:
r_and_d_dd = get_dropdown_options(spreadsheet_id, sheet_name, 'B16')
operating_lease_dd = get_dropdown_options(spreadsheet_id, sheet_name, 'B17')

In [ ]:
r_and_d = input('Do you have R&D expenses to capitalize?')
operating_lease_commitments = input('Do you have operating lease commitments?')
input_worksheet.update('B16', [[r_and_d]])
input_worksheet.update('B17', [[operating_lease_commitments]])

## Cash and Marketable Securities (Most Recent 12 Months, Last 10K)

In [ ]:
current_cash_and_marketable_securities = input('What is the current cash and marketable securities?')
last_10k_cash_and_marketable_securities = input('What is the last 10K cash and marketable securities?')
input_worksheet.update('B18', [[current_cash_and_marketable_securities]])
input_worksheet.update('C18', [[last_10k_cash_and_marketable_securities]])

## Cross Holdings and Other Non-Operating Assets (Most Recent 12 Months, Last 10K)

In [ ]:
cross_holdings_and_other_non_operating_assets = input('What is the current cross holdings and other non-operating assets?')
last_10k_cross_holdings_and_other_non_operating_assets = input('What is the last 10K cross holdings and other non-operating assets?')
input_worksheet.update('B19', [[cross_holdings_and_other_non_operating_assets]])
input_worksheet.update('C19', [[last_10k_cross_holdings_and_other_non_operating_assets]])

## Minority Interests (Most Recent 12 Months, Last 10K)

In [ ]:
current_minority_interests = input('What is the current minority interests?')
last_10k_minority_interests = input('What is the last 10K minority interests?')
input_worksheet.update('B20', [[current_minority_interests]])
input_worksheet.update('C20', [[last_10k_minority_interests]])

## Number of Shares Outstanding, Current Stock Price, Effect and Marginal Tax Rate

In [ ]:
number_of_shares_outstanding = input('What is the number of shares outstanding?')
current_stock_price = input('What is the current stock price?')
effective_tax_rate = input('What is the effective tax rate?')
marginal_tax_rate = input('What is the marginal tax rate?')
input_worksheet.update('B21', [[number_of_shares_outstanding]])
input_worksheet.update('B22', [[current_stock_price]])
input_worksheet.update('B23', [[effective_tax_rate]])
input_worksheet.update('B24', [[marginal_tax_rate]])

## Value Drivers

In [ ]:
rev_growth_rate_for_next_year = input('What is the revenue growth rate for the next year?')
operating_margin_for_next_year = input('What is the operating margin for the next year?')
compounded_annual_growth_rate = input('What is the compounded annual growth rate?')
target_pre_tax_operating_margin = input('What is the target pre-tax operating margin?')
years_of_convergence_for_margin = input('How many years of convergence for the margin?')
sales_to_capital_ratio_for_years_1_to_5 = input('What is the sales to capital ratio for the first 5 years?')
sales_to_capital_ratio_for_years_6_to_10 = input('What is the sales to capital ratio for the last 5 years?')
input_worksheet.update('B26', [[rev_growth_rate_for_next_year]])
input_worksheet.update('B27', [[operating_margin_for_next_year]])
input_worksheet.update('B28', [[compounded_annual_growth_rate]])
input_worksheet.update('B29', [[target_pre_tax_operating_margin]])
input_worksheet.update('B30', [[years_of_convergence_for_margin]])
input_worksheet.update('B31', [[sales_to_capital_ratio_for_years_1_to_5]])
input_worksheet.update('B32', [[sales_to_capital_ratio_for_years_6_to_10]])

## Market Numbers

In [ ]:
rf = input('What is the riskfree rate?')
input_worksheet.update('B33', [[rf]])

## Other Inputs

In [ ]:
employee_options_outstanding = input('Do you have employee options outstanding?')
number_of_options_outstanding = input('What is the number of options outstanding?')
average_strike_price = input('What is the average strike price?')
average_maturity = input('What is the average maturity?')
standard_deviation_on_stock_price = input('What is the standard deviation on the stock price?')
input_worksheet.update('B37', [[employee_options_outstanding]])
input_worksheet.update('B38', [[number_of_options_outstanding]])
input_worksheet.update('B39', [[average_strike_price]])
input_worksheet.update('B40', [[average_maturity]])
input_worksheet.update('B41', [[standard_deviation_on_stock_price]])

In [ ]:
valuation_worksheet.acell('B33').value